In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

In [11]:
data_df = pd.read_csv("data/final_dataset.csv")

data_df = data_df.drop(['Date'], axis=1)
data_df.head()

,Open,High,Low,Close,MACD,ATR,RSI,EFFR,VIX,USDX,UNRATE,UMCSENT
0,1498.13,1508.18,1497.56,1498.13,-3.311325,17.195714,57.426836,NaN,17.469999,111.790001,4.1,107.3
1,1505.97,1507.30,1489.39,1505.97,-2.269902,17.627857,60.150376,NaN,17.379999,111.419998,4.1,107.3
2,1508.31,1511.16,1501.30,1508.31,-1.556307,17.228571,63.012140,NaN,17.040001,111.230003,4.1,107.3
3,1506.45,1513.41,1505.16,1506.45,-0.873683,18.557143,67.127444,NaN,16.530001,111.349998,4.1,107.3
4,1514.09,1523.88,1506.46,1514.09,0.124412,20.516429,76.296296,NaN,16.540001,111.459999,4.1,107.3


In [12]:
# data_df.index
data_df = data_df.iloc[0:200,:]
data_df.index
data = np.array(data_df, dtype=np.float32)
# replace nans with 0
data = np.nan_to_num(data)
print(data)

[[1498.13 1508.18 1497.56 ...  111.79    4.1   107.3 ]
 [1505.97 1507.3  1489.39 ...  111.42    4.1   107.3 ]
 [1508.31 1511.16 1501.3  ...  111.23    4.1   107.3 ]
 ...
 [1283.57 1286.62 1267.11 ...  118.7     4.5    92.6 ]
 [1270.03 1283.85 1269.03 ...  119.36    4.5    92.6 ]
 [1276.96 1277.08 1265.08 ...  119.09    4.5    92.6 ]]


In [13]:
def scale_data(data):
    num_features = data.shape[1]
    scale_params = np.zeros((num_features,2))
    for i in range(num_features):
        min_val = np.min(data[:,i])
        max_val = np.max(data[:,i])
        scale_params[i,:] = [min_val, max_val]
        data[:,i] = (data[:,i] - min_val) / (max_val - min_val)
    return data, scale_params

data, scale_params = scale_data(data)
print(data.shape)

(200, 12)


/tmp/ipykernel_22525/306952755.py:8: RuntimeWarning: invalid value encountered in true_divide
  data[:,i] = (data[:,i] - min_val) / (max_val - min_val)


In [14]:
def create_sequence(data,seq_len):
    xs = []
    ys = []
    for i in range(len(data)-seq_len-1):
        x = data[i:(i+seq_len),:]
        # print(x.shape)
        y = data[i+seq_len,0]
        xs.append(x)
        ys.append(y)
    return np.array(xs),np.array(ys)

inputs , targets = create_sequence(data,5)
print(inputs.shape)
print(targets.shape)

(194, 5, 12)
(194,)


In [15]:
# can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32,
print(inputs.dtype)
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


dataset = TensorDataset(inputs,targets)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset) - int(0.8*len(dataset))])

train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)

class LSTM(nn.Module):
    def __init__(self, input_size=12, hidden_layer_size=30, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size,batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.cell_double = (torch.zeros(1,self.hidden_layer_size),
                        torch.zeros(1,self.hidden_layer_size))
    def forward(self, input_seq):
        print("here")
        lstm_out, self.cell_double = self.lstm(input_seq, self.cell_double)
        predictions = self.linear(lstm_out)
        print(predictions)
        return predictions[-1]


float32


In [17]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 194 
for i in range(epochs):
    model.cell_double = (torch.zeros(1,model.hidden_layer_size),
                        torch.zeros(1,model.hidden_layer_size))
    for seq, labels in train_loader:
        seq = seq.view(5,12)
        optimizer.zero_grad()
        y_pred = model(seq)
        print(y_pred)
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    # if(i%5==0):

    #     print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

here
tensor([nan], grad_fn=<SelectBackward0>)
here
tensor([nan], grad_fn=<SelectBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.